In [42]:
from parse_crystfel_stream import parse_crystfel_stream

stream_file = "/home/bubl3932/files/UOX_sim/simulation-10/xgandalf_iterations_max_radius_1.8_step_0.5/UOXsim_0.0_0.0.stream"

df_refl, df_cryst = parse_crystfel_stream(stream_file, debug=True)

Parsed 1204281 reflection rows total.
Parsed 387 crystal rows (events).


In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Suppose you already have df_refl, df_cryst from parse_crystfel_stream

# Each row of df_cryst has columns like:
#  event, astar, bstar, cstar, a, b, c, alpha, beta, gamma
# where astar, bstar, cstar are each [vx, vy, vz].
# We'll expand these into individual columns for easier merging and computation.

def expand_vector_columns(df, colname):
    """
    df[colname] is a list/tuple of 3 floats [vx, vy, vz].
    We'll expand them into colname+'_x', colname+'_y', colname+'_z'
    """
    return pd.DataFrame(
        df[colname].tolist(), 
        index=df.index, 
        columns=[f"{colname}_x", f"{colname}_y", f"{colname}_z"]
    )

# Make copies so we don't overwrite the original
df_cryst_expanded = df_cryst.copy()

for vec in ["astar", "bstar", "cstar"]:
    if vec in df_cryst_expanded.columns:
        # Expand into separate columns
        expanded = expand_vector_columns(df_cryst_expanded, vec)
        # Add to df_cryst_expanded
        for c in expanded.columns:
            df_cryst_expanded[c] = expanded[c]
        # Drop the original vector column
        df_cryst_expanded.drop(columns=[vec], inplace=True, errors="ignore")

# Now df_cryst_expanded has columns:
#   ['event', 'a', 'b', 'c', 'alpha', 'beta', 'gamma',
#    'astar_x','astar_y','astar_z', 'bstar_x','bstar_y','bstar_z',
#    'cstar_x','cstar_y','cstar_z']

# Merge reflection data with crystal data on 'event'
df_merged = pd.merge(df_refl, df_cryst_expanded, on='event', how='left')

df_merged.head()


,event,h,k,l,I,sigma_I,peak,background,fs_px,ss_px,...,gamma,astar_x,astar_y,astar_z,bstar_x,bstar_y,bstar_z,cstar_x,cstar_y,cstar_z
0,0,-43,-71,-54,-1.62,6.14,13.38,0.12,14.9,19.2,...,89.81227,-0.000389,0.087842,0.085035,0.105752,-2.000000e-07,1.000000e-07,0.0003,0.06789,-0.06597
1,0,-43,-69,-54,129.79,25.54,22.06,0.06,28.9,19.2,...,89.81227,-0.000389,0.087842,0.085035,0.105752,-2.000000e-07,1.000000e-07,0.0003,0.06789,-0.06597
2,0,-43,-67,-54,267.79,36.59,45.03,0.00,42.9,19.2,...,89.81227,-0.000389,0.087842,0.085035,0.105752,-2.000000e-07,1.000000e-07,0.0003,0.06789,-0.06597
3,0,-43,-65,-54,652.39,57.11,109.14,0.00,56.9,19.2,...,89.81227,-0.000389,0.087842,0.085035,0.105752,-2.000000e-07,1.000000e-07,0.0003,0.06789,-0.06597
4,0,-43,-63,-54,1379.49,83.05,229.58,0.00,70.9,19.2,...,89.81227,-0.000389,0.087842,0.085035,0.105752,-2.000000e-07,1.000000e-07,0.0003,0.06789,-0.06597


In [63]:
import matplotlib.pyplot as plt

def plot_intensity_histogram(df_merged, asu_idx, bins=50):
    """
    Plot a histogram of intensities for all reflections that map to the specified symmetry group.
    
    Parameters:
        df_merged (pd.DataFrame): The merged DataFrame that includes an "asu_index" column and an intensity "I" column.
        asu_idx (tuple): The symmetry group identifier (e.g., (h_asu, k_asu, l_asu)).
        bins (int): Number of bins for the histogram.
    """
    # Filter the DataFrame for rows that belong to the given symmetry group
    group = df_merged[df_merged["asu_index"] == asu_idx]
    
    if group.empty:
        print(f"No data found for symmetry group {asu_idx}")
        return

    # Extract the intensity values from the group
    intensities = group["I"].values

    # Create the histogram
    plt.figure(figsize=(8, 6))
    plt.hist(intensities, bins=bins, color="skyblue", edgecolor="black")
    plt.xlabel("Intensity")
    plt.ylabel("Frequency")
    plt.title(f"Intensity Histogram for Symmetry Group {asu_idx}")
    plt.tight_layout()
    plt.show()

# Example usage:
# Replace (3, 2, 1) with the desired asu_index from your DataFrame.
plot_intensity_histogram(df_merged, asu_idx=(1, 2, 1), bins=50)


In [44]:
grouped = df_merged.groupby(["h", "k", "l"])


In [45]:
# Group by the reflection indices and count the number of occurrences
common_refl = df_merged.groupby(["h", "k", "l"]).size().reset_index(name="count")

# Sort the results by count in descending order
common_refl.sort_values("count", ascending=False, inplace=True)

# Display the top 10 most common reflections
print(common_refl.head(10))


        h  k  l  count
352650  0  1 -1     51
352499  0 -1  1     51
352498  0 -1 -1     49
352651  0  1  1     49
342927 -1 -1  0     46
360243  1  1  0     46
352729  0  2  0     41
360115  1 -1  0     41
343040 -1  1  0     41
352574  0  0 -2     40


In [46]:
import matplotlib.pyplot as plt

# Create a new column with a string representation of the reflection for labeling
common_refl["reflection"] = common_refl.apply(lambda row: f"({row['h']},{row['k']},{row['l']})", axis=1)

# Plot the top 10 most common reflections
top_refs = common_refl.head(100)
plt.figure(figsize=(10, 6))
plt.bar(top_refs["reflection"], top_refs["count"], color="skyblue")
plt.xlabel("Reflection (h,k,l)")
plt.ylabel("Count")
plt.title("Top 10 Most Common Reflections")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [15]:
# Example: get group for reflection h=1, k=2, l=3
reflection_123 = grouped.get_group((6, -6, 0))

print(reflection_123)


           event  h  k  l         I  sigma_I    peak  background  fs_px  \
428956     814-1  6 -6  0  19800.17   851.08  3577.0      589.22  537.3   
511139     994-1  6 -6  0   7115.20   591.74  2233.0      520.75  500.9   
536050    1009-1  6 -6  0    276.35   334.09   660.0      472.59  520.3   
627838    1122-1  6 -6  0   1027.44  1001.59  1706.0      750.35  502.5   
726327    1287-2  6 -6  0    358.54   332.27   596.0      392.96  559.5   
...          ... .. .. ..       ...      ...     ...         ...    ...   
8881314  10333-2  6 -6  0  27809.46   994.56  3965.0      721.81  496.3   
8950389  10404-2  6 -6  0  66922.52  1328.88  8896.0      686.81  547.7   
8955149  10405-2  6 -6  0   3510.94   339.57  1109.0      328.47  516.0   
9285336  10798-1  6 -6  0  10501.12   651.08  2520.0      572.22  509.8   
9488447  11103-1  6 -6  0  14115.77  1075.07  3473.0      747.33  486.7   

         ss_px  ...   cstar_x   cstar_y   cstar_z        Qx        Qy  \
428956   548.2  ... -0.032

In [47]:
# Compute the Qx, Qy, Qz columns vectorized:
df_merged["Qx"] = df_merged["h"] * df_merged["astar_x"] + \
                   df_merged["k"] * df_merged["bstar_x"] + \
                   df_merged["l"] * df_merged["cstar_x"]

df_merged["Qy"] = df_merged["h"] * df_merged["astar_y"] + \
                   df_merged["k"] * df_merged["bstar_y"] + \
                   df_merged["l"] * df_merged["cstar_y"]

df_merged["Qz"] = df_merged["h"] * df_merged["astar_z"] + \
                   df_merged["k"] * df_merged["bstar_z"] + \
                   df_merged["l"] * df_merged["cstar_z"]

# Compute the norm in a vectorized way:
df_merged["Q_norm"] = np.sqrt(df_merged["Qx"]**2 + df_merged["Qy"]**2 + df_merged["Qz"]**2) + 1e-30

# Normalize to get unit vectors:
df_merged["Qx_unit"] = df_merged["Qx"] / df_merged["Q_norm"]
df_merged["Qy_unit"] = df_merged["Qy"] / df_merged["Q_norm"]
df_merged["Qz_unit"] = df_merged["Qz"] / df_merged["Q_norm"]


In [61]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

def plot_spherical_heatmap(df_merged, h, k, l, n_theta=36, n_phi=72):
    """
    Create a spherical heatmap of the reflection intensity for reflection (h, k, l).
    
    Steps:
      1) Extract all measured (x,y,z) directions on the unit sphere for (h,k,l).
      2) Convert each to spherical angles (theta, phi).
      3) Bin the angles -> compute average intensity in each bin.
      4) Plot a 3D sphere, coloring each bin's surface patch by avg intensity.
    
    Arguments:
        df_merged: Your merged DataFrame, which has columns:
                   'Qx_unit', 'Qy_unit', 'Qz_unit' (unit vectors) and 'I' (intensities).
        h, k, l:   Miller indices of the reflection you want to visualize.
        n_theta:   Number of bins in the polar/inclination direction (0 to pi).
        n_phi:     Number of bins in the azimuthal direction (0 to 2*pi).
    """
    # Subset for the chosen reflection
    group = df_merged[(df_merged["h"] == h) &
                      (df_merged["k"] == k) &
                      (df_merged["l"] == l)]
    if len(group) == 0:
        print(f"No data found for reflection ({h},{k},{l})")
        return

    # Extract unit-sphere coords and intensities
    x = group["Qx_unit"].values
    y = group["Qy_unit"].values
    z = group["Qz_unit"].values
    intensities = group["I"].values

    # Convert each (x,y,z) to spherical angles:
    #   phi   in [0, 2*pi],   phi=0 along +x, increases toward +y
    #   theta in [0,   pi ],  theta=0 at +z, increases down to -z
    phi = np.arctan2(y, x)            # range -π to +π
    phi[phi < 0] += 2*np.pi           # shift to [0, 2π)
    theta = np.arccos(z)             # range [0, π]

    # Create bin edges in theta and phi
    theta_bins = np.linspace(0, np.pi, n_theta+1)
    phi_bins = np.linspace(0, 2*np.pi, n_phi+1)

    # 2D histogram for "counts" in each (theta, phi) bin
    # 2D histogram for "sum of intensities" in each (theta, phi) bin
    counts, _, _ = np.histogram2d(theta, phi, bins=(theta_bins, phi_bins))
    sums, _, _ = np.histogram2d(theta, phi, bins=(theta_bins, phi_bins),
                                weights=intensities)

    # Average intensity per bin
    avg_intensity = sums / np.maximum(counts, 1)  # avoid divide-by-zero

    # For plotting, we build a mesh at the centers of these bins
    #   Phi  in shape (n_theta, n_phi)
    #   Theta in shape (n_theta, n_phi)
    theta_centers = 0.5 * (theta_bins[:-1] + theta_bins[1:])
    phi_centers   = 0.5 * (phi_bins[:-1]   + phi_bins[1:])
    Theta, Phi    = np.meshgrid(theta_centers, phi_centers, indexing='ij')

    # Convert back to Cartesian on the unit sphere
    X = np.sin(Theta) * np.cos(Phi)
    Y = np.sin(Theta) * np.sin(Phi)
    Z = np.cos(Theta)

    # avg_intensity is shape (n_theta, n_phi),
    # but our mesh is (n_theta, n_phi) => they match.
    # We'll normalize intensities so we can apply a colormap.
    # (If all intensities are identical, the colormap can appear uniform.)
    i_min, i_max = avg_intensity.min(), avg_intensity.max()
    # If i_max == i_min, everything is the same intensity.
    denom = i_max - i_min if (i_max > i_min) else 1e-30
    normed = (avg_intensity - i_min) / denom

    # Plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # We'll use plot_surface with facecolors mapped from normed intensities.
    # E.g., use "viridis" colormap; you can choose any built-in cmap you like.
    facecolors = plt.cm.viridis(normed)

    surf = ax.plot_surface(X, Y, Z, facecolors=facecolors,
                           rcount=n_theta, ccount=n_phi)

    # Add a colorbar. We create a ScalarMappable just for the color scale.
    mappable = plt.cm.ScalarMappable(cmap=plt.cm.viridis)
    mappable.set_array(avg_intensity)
    cbar = plt.colorbar(mappable, ax=ax)
    cbar.set_label("Average Intensity")

    ax.set_title(f"Spherical Heatmap for Reflection ({h},{k},{l})")
    # Optionally, turn off axis lines:
    # ax.set_axis_off()
    plt.show()

# Example usage
plot_spherical_heatmap(df_merged, h=1, k=2, l=1, n_theta=36, n_phi=72)


In [51]:
from cctbx import crystal, miller
from cctbx.array_family import flex

# Example placeholders (replace with your actual cell):
# unit_cell = (87.4, 87.4, 87.4, 90.0, 90.0, 90.0)
# space_group_symbol = "Fm-3m"  # or any other, e.g. "P212121"
unit_cell = (80.58, 94.49, 103.89, 90.0, 90.0, 90.0)
space_group_symbol = "I222"  # or any other, e.g. "P212121"

cryst_sym = crystal.symmetry(
    unit_cell=unit_cell,
    space_group_symbol=space_group_symbol
)

# all_indices is a list of (h,k,l) from your DataFrame
all_indices = list(zip(df_merged["h"], df_merged["k"], df_merged["l"]))

all_indices_flex = flex.miller_index(all_indices)
ms = miller.set(
    crystal_symmetry=cryst_sym,
    indices=all_indices_flex,
    anomalous_flag=False  # or True if relevant
)

ms_asu = ms.map_to_asu()
mapped_indices = list(ms_asu.indices())  # cctbx 'asu' index

# Attach these to df_merged
df_merged["asu_index"] = mapped_indices


In [52]:
grouped_asu = df_merged.groupby("asu_index")


In [53]:
# Count how many reflections fall in each symmetry group
counts = grouped_asu.size().reset_index(name="count")
counts.sort_values("count", ascending=False, inplace=True)
print(counts.head(10))


      asu_index  count
3088  (1, 2, 1)    208
39    (0, 1, 1)    200
3049  (1, 1, 2)    185
6163  (2, 1, 1)    176
3048  (1, 1, 0)    174
6203  (2, 2, 2)    155
3123  (1, 3, 2)    153
6164  (2, 1, 3)    142
3162  (1, 4, 1)    138
3089  (1, 2, 3)    130


In [64]:
import matplotlib.pyplot as plt

def plot_intensity_histogram(df_merged, asu_idx, bins=50):
    """
    Plot a histogram of intensities for all reflections that map to the specified symmetry group.
    
    Parameters:
        df_merged (pd.DataFrame): The merged DataFrame that includes an "asu_index" column and an intensity "I" column.
        asu_idx (tuple): The symmetry group identifier (e.g., (h_asu, k_asu, l_asu)).
        bins (int): Number of bins for the histogram.
    """
    # Filter the DataFrame for rows that belong to the given symmetry group
    group = df_merged[df_merged["asu_index"] == asu_idx]
    
    if group.empty:
        print(f"No data found for symmetry group {asu_idx}")
        return

    # Extract the intensity values from the group
    intensities = group["I"].values

    # Create the histogram
    plt.figure(figsize=(8, 6))
    plt.hist(intensities, bins=bins, color="skyblue", edgecolor="black")
    plt.xlabel("Intensity")
    plt.ylabel("Frequency")
    plt.title(f"Intensity Histogram for Symmetry Group {asu_idx}")
    plt.tight_layout()
    plt.show()

# Example usage:
# Replace (3, 2, 1) with the desired asu_index from your DataFrame.
plot_intensity_histogram(df_merged, asu_idx=(0, 1, 1), bins=50)


In [54]:
import numpy as np
import matplotlib.pyplot as plt

def plot_spherical_heatmap_asu(df_merged, asu_idx, n_theta=36, n_phi=72):
    """
    Create a spherical heatmap of the reflection intensity for ALL reflections
    that map to a particular ASU index (h_asu, k_asu, l_asu). This lumps together
    all symmetry-equivalent (h,k,l).
    
    Steps:
      1) Extract all measured (x,y,z) directions on the unit sphere for that asu_idx.
      2) Convert each to spherical angles (theta, phi).
      3) Bin the angles -> compute average intensity in each bin.
      4) Plot a 3D sphere, coloring each bin's surface patch by avg intensity.
    """
    # Select the subset of data for this group
    group = df_merged[df_merged["asu_index"] == asu_idx]
    
    if len(group) == 0:
        print(f"No data found for asu_index {asu_idx}")
        return

    # Extract unit-sphere coords and intensities
    x = group["Qx_unit"].values
    y = group["Qy_unit"].values
    z = group["Qz_unit"].values
    intensities = group["I"].values

    # Convert (x,y,z) to spherical angles
    phi = np.arctan2(y, x)  # range -π to +π
    phi[phi < 0] += 2*np.pi  # shift to [0, 2π)
    theta = np.arccos(z)     # range [0, π]

    # Bin edges
    theta_bins = np.linspace(0, np.pi, n_theta + 1)
    phi_bins   = np.linspace(0, 2*np.pi, n_phi + 1)

    # 2D histograms: counts and sums
    counts, _, _ = np.histogram2d(theta, phi, bins=(theta_bins, phi_bins))
    sums,   _, _ = np.histogram2d(theta, phi, bins=(theta_bins, phi_bins),
                                  weights=intensities)

    avg_intensity = sums / np.maximum(counts, 1)

    # Build grid for plotting
    theta_centers = 0.5*(theta_bins[:-1] + theta_bins[1:])
    phi_centers   = 0.5*(phi_bins[:-1]   + phi_bins[1:])
    Theta, Phi    = np.meshgrid(theta_centers, phi_centers, indexing='ij')

    # Map spherical -> Cartesian
    X = np.sin(Theta) * np.cos(Phi)
    Y = np.sin(Theta) * np.sin(Phi)
    Z = np.cos(Theta)

    # Normalize intensities for colormap
    i_min, i_max = avg_intensity.min(), avg_intensity.max()
    denom = (i_max - i_min) if (i_max > i_min) else 1e-30
    normed = (avg_intensity - i_min) / denom

    # 3D plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    facecolors = plt.cm.viridis(normed)
    surf = ax.plot_surface(X, Y, Z, facecolors=facecolors,
                           rcount=n_theta, ccount=n_phi)

    # Colorbar
    mappable = plt.cm.ScalarMappable(cmap=plt.cm.viridis)
    mappable.set_array(avg_intensity)
    cbar = plt.colorbar(mappable, ax=ax)
    cbar.set_label("Average Intensity")

    ax.set_title(f"Spherical Heatmap for ASU index {asu_idx}")
    plt.show()


In [60]:
plot_spherical_heatmap_asu(df_merged, asu_idx=(0,1,1), n_theta=36, n_phi=72)
